In [176]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import StandardScaler

In [177]:
dados = pd.read_csv(
    r"D:\deltalakeapps\excelfiles\vendas2008.csv"
    ,sep=";")
dados.dropna(inplace=True)

In [178]:
dados.dtypes

vl_orcamento         float64
id_quote              object
convertido             int64
periodo_mes            int64
id_especificador      object
ds_tipo_obra          object
promocional            int64
qt_itens               int64
vl_orcamento.1       float64
vl_frete             float64
REVESTIMENTO          object
LOUCAS_METAIS         object
ARGAMASSA_REJUNTE     object
dtype: object

In [179]:
dados.head(3)

,vl_orcamento,id_quote,convertido,periodo_mes,id_especificador,ds_tipo_obra,promocional,qt_itens,vl_orcamento.1,vl_frete,REVESTIMENTO,LOUCAS_METAIS,ARGAMASSA_REJUNTE
0,15679.40,0Q03s000000GRo9CAG,1,1,0013s000011bE6KAAU,Reforma,1,3,15679.40,300.0,S,N,S
1,3946.63,0Q03s000000GRoYCAW,1,1,0013s000018c3eaAAA,Reforma,1,1,3946.63,125.0,S,N,N
2,2144.96,0Q03s000000GRodCAG,1,1,0011I00000UeRIlQAN,Nova Obra,1,1,2144.96,0.0,S,N,N


In [180]:
dados.columns

Index(['vl_orcamento', 'id_quote', 'convertido', 'periodo_mes',
       'id_especificador', 'ds_tipo_obra', 'promocional', 'qt_itens',
       'vl_orcamento.1', 'vl_frete', 'REVESTIMENTO', 'LOUCAS_METAIS',
       'ARGAMASSA_REJUNTE'],
      dtype='object')

In [181]:

dados.columns = dados.columns.str.lower()

dados.rename(columns={
    'vl_orcamento': 'vl_orcamento',
    'id_quote': 'id_quote',
    'convertido': 'convertido',
    'periodo_mes': 'periodo_mes',
    'id_especificador': 'id_especificador',
    'ds_tipo_obra': 'ds_tipo_obra',
    'promocional': 'promocional',
    'qt_itens': 'qt_itens',
    'vl_orcamento.1': 'vl_orcamento_1',
    'vl_frete': 'vl_frete',
    'REVESTIMENTO': 'revestimento',
    'LOUCAS_METAIS': 'loucas_metais',
    'ARGAMASSA_REJUNTE': 'argamassa_rejunte'
}, inplace=True)


In [182]:
count_ds_tipo_obra = dados['ds_tipo_obra'].value_counts().reset_index()
count_ds_tipo_obra

,ds_tipo_obra,count
0,Reforma,43
1,Nova Obra,35


In [183]:
count_ds_tipo_obra.columns = ['ds_tipo_obra', 'ds_tipo_obra_count']

df = dados.merge(count_ds_tipo_obra, on='ds_tipo_obra', how='left')

In [184]:
df

,vl_orcamento,id_quote,convertido,periodo_mes,id_especificador,ds_tipo_obra,promocional,qt_itens,vl_orcamento_1,vl_frete,revestimento,loucas_metais,argamassa_rejunte,ds_tipo_obra_count
0,15679.40,0Q03s000000GRo9CAG,1,1,0013s000011bE6KAAU,Reforma,1,3,15679.40,300.00,S,N,S,43
1,3946.63,0Q03s000000GRoYCAW,1,1,0013s000018c3eaAAA,Reforma,1,1,3946.63,125.00,S,N,N,43
2,2144.96,0Q03s000000GRodCAG,1,1,0011I00000UeRIlQAN,Nova Obra,1,1,2144.96,0.00,S,N,N,35
3,15010.43,0Q03s000000GRs1CAG,1,1,0013s00001A6i3MAAR,Nova Obra,1,5,15010.43,550.00,S,N,N,35
4,2620.77,0Q03s000000GRs6CAG,1,1,0011I00000UeF6FQAV,Reforma,1,1,2620.77,0.00,S,N,N,43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,2745.61,0Q03s000000GSZUCA14,1,1,0011I00000UgtLHQAZ,Nova Obra,1,97,2745.61,0.10,S,N,N,35
74,1930.26,0Q03s000000GSaICAW,1,1,0011I00000v32aZQAQ,Reforma,1,101,1930.26,150.08,S,N,S,43
75,2745.60,0Q03s000000GSZUCA13,1,1,0011I00000UgtLHQAZ,Nova Obra,1,105,2745.60,0.09,S,N,N,35
76,1930.27,0Q03s000000GSaICAW,1,1,0011I00000v32aZQAQ,Reforma,1,109,1930.27,150.09,S,N,S,43


In [185]:

categorical_columns = df.select_dtypes(include=[object]).columns

onehot_encoder = OneHotEncoder(sparse=False, drop='first')
onehot_encoded = onehot_encoder.fit_transform(df[categorical_columns])


onehot_df = pd.DataFrame(onehot_encoded, columns=onehot_encoder.get_feature_names_out(categorical_columns))


df = df.drop(columns=categorical_columns)


df = pd.concat([df, onehot_df], axis=1)
df


d:\deltalakeapps\venv\Lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,vl_orcamento,convertido,periodo_mes,promocional,qt_itens,vl_orcamento_1,vl_frete,ds_tipo_obra_count,id_quote_0Q03s000000GRoYCAW,id_quote_0Q03s000000GRodCAG,...,id_especificador_0013s00001AwfPmAAJ,id_especificador_0013s00001BSMUhAAP,id_especificador_0013s00001BSu2iAAD,id_especificador_0013s00001D2eAxAAJ,id_especificador_0013s00001Dk43NAAR,id_especificador_0013s00001DkWqxAAF,ds_tipo_obra_Reforma,revestimento_S,loucas_metais_S,argamassa_rejunte_S
0,15679.40,1,1,1,3,15679.40,300.00,43,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
1,3946.63,1,1,1,1,3946.63,125.00,43,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
2,2144.96,1,1,1,1,2144.96,0.00,35,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,15010.43,1,1,1,5,15010.43,550.00,35,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,2620.77,1,1,1,1,2620.77,0.00,43,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,2745.61,1,1,1,97,2745.61,0.10,35,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
74,1930.26,1,1,1,101,1930.26,150.08,43,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
75,2745.60,1,1,1,105,2745.60,0.09,35,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
76,1930.27,1,1,1,109,1930.27,150.09,43,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0


In [186]:


categorical_columns = df.select_dtypes(include=[object]).columns

onehot_encoder = OneHotEncoder(sparse=False, drop='first')
onehot_encoded = onehot_encoder.fit_transform(df[categorical_columns])

onehot_df = pd.DataFrame(onehot_encoded, columns=onehot_encoder.get_feature_names_out(categorical_columns))

df = df.drop(columns=categorical_columns)
df = pd.concat([df, onehot_df], axis=1)


X = df.drop(columns=['convertido'])
y = df['convertido']

"""dados  (80% treinamento, 20% teste)"""
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


model = keras.Sequential()
model.add(layers.Dense(128, activation='relu', input_dim=X_train.shape[1]))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2)

y_prob = model.predict(X_test)


results = pd.DataFrame({
    'convertido': y_test,
    'promocional': df.loc[y_test.index, 'promocional'],
    'ds_tipo_obra_count': df.loc[y_test.index, 'ds_tipo_obra_count'],
    'Probabilidade_Convertido_1': y_prob.flatten()
})

valor = results.columns = ['convertido', 'promocional', 'ds_tipo_obra_count', 'Probabilidade_Convertido_1']

print(results)



Epoch 1/100


d:\deltalakeapps\venv\Lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


2/2 [==============================] - 1s 170ms/step - loss: 0.8812 - accuracy: 0.2449 - val_loss: 0.6298 - val_accuracy: 0.8462
Epoch 2/100
2/2 [==============================] - 0s 31ms/step - loss: 0.5265 - accuracy: 0.8980 - val_loss: 0.4191 - val_accuracy: 1.0000
Epoch 3/100
2/2 [==============================] - 0s 28ms/step - loss: 0.3154 - accuracy: 1.0000 - val_loss: 0.2808 - val_accuracy: 1.0000
Epoch 4/100
2/2 [==============================] - 0s 26ms/step - loss: 0.1944 - accuracy: 1.0000 - val_loss: 0.1908 - val_accuracy: 1.0000
Epoch 5/100
2/2 [==============================] - 0s 32ms/step - loss: 0.1242 - accuracy: 1.0000 - val_loss: 0.1323 - val_accuracy: 1.0000
Epoch 6/100
2/2 [==============================] - 0s 30ms/step - loss: 0.0816 - accuracy: 1.0000 - val_loss: 0.0941 - val_accuracy: 1.0000
Epoch 7/100
2/2 [==============================] - 0s 30ms/step - loss: 0.0562 - accuracy: 1.0000 - val_loss: 0.0686 - val_accuracy: 1.0000
Epoch 8/100
2/2 [==============

In [187]:
results

,convertido,promocional,ds_tipo_obra_count,Probabilidade_Convertido_1
33,1,1,43,0.999950
0,1,1,43,0.973518
34,1,1,43,0.963003
12,1,1,35,0.985362
10,1,1,43,0.991151
77,1,1,35,0.999912
30,1,1,35,0.983405
4,1,1,43,0.978498
49,1,1,35,0.990267
66,1,1,43,0.999528
